In [ ]:
%%configure -f
{
  "conf": {
    "spark.notebook.parameters": "{}"
  },
  "defaultLakehouse": {
    "name": "BenchmarkLakehouse"
  }
}


# 📓2. Apply Updates
### 💤 Wake up Azure SQL before running.

Happy‑path notebook that implements three strategies: Full Refresh, Full Compare (append events), and Incremental (append updates).
This notebook follows the ingest_data conventions for parameter handling and metrics. It expects the deployment tooling to populate the %%configure cell with the parameter set (see provision_notebooks.py).
Style: minimal, linear, assert preconditions, debug prints, let errors surface.

In [ ]:
# Imports and small helpers used across cells
import json
import time
from datetime import datetime
import os
from pathlib import Path
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
# synapsesql needs this import to work
from com.microsoft.spark.fabric import Constants
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, FloatType
print('Imports ready')


In [ ]:
# Helper: make specified columns nullable (used for warehouse writes to match schema expectations)
from pyspark.sql.types import StructType, StructField
def make_columns_nullable(df, columns=None):
    if columns is None:
        columns = [f.name for f in df.schema.fields]
    new_schema = StructType([
        StructField(f.name, f.dataType, True) if f.name in columns else f
        for f in df.schema.fields
    ])
    return df.sparkSession.createDataFrame(df.rdd, schema=new_schema)
print('make_columns_nullable ready')


In [ ]:
# Parameters and global variables (happy-path, assert presence)
spark = SparkSession.builder.getOrCreate()
conf_key = 'spark.notebook.parameters'
conf_str = None
try:
    conf_str = spark.conf.get(conf_key, None)
except Exception:
    conf_str = None
if not conf_str:
    try:
        conf_str = spark.sparkContext.getConf().get(conf_key, None)
    except Exception:
        conf_str = None
if not conf_str:
    raise SystemExit('Missing spark.notebook.parameters. Provisioning/dev should populate the cell before run.')

params = json.loads(conf_str)

# Minimal expected keys; we assert existence but do not defensively default other than optional AZURE_SQL_* fields inserted by provisioner
required = ['name', 'dataset_name', 'source', 'format', 'update_strategy']
missing = [k for k in required if k not in params]
if missing:
    raise SystemExit(f"Missing required parameters: {', '.join(missing)}")

test_case_name = params['name']
dataset_name = params['dataset_name']
source = str(params['source']).lower()   # 'lakehouse' or 'sql'
fmt = str(params['format']).lower()      # 'delta' or 'warehouse'
update_strategy = str(params['update_strategy']).lower()  # 'full refresh', 'full compare', 'incremental'
notes = params.get('notes', '')

# SQL connection info if present
AZURE_SQL_SERVER = params.get('AZURE_SQL_SERVER')
AZURE_SQL_DB = params.get('AZURE_SQL_DB')
AZURE_SQL_SCHEMA = params.get('AZURE_SQL_SCHEMA', 'dbo')

# Reuse naming logic from ingest_data: sanitized target name
import re
sanitized_name = re.sub(r"[^a-z0-9_]", "", re.sub(r"[\s-]+", "_", test_case_name.strip().lower()))
target_table = sanitized_name

# Use the controller (BFF-Controller) DataSourceLakehouse ABFSS paths for source data (match ingest_data pattern)
controller_workspace_name = 'BFF-Controller'
controller_lakehouse_name = 'DataSourceLakehouse'
abfss_account = 'onelake.dfs.fabric.microsoft.com'
container = controller_workspace_name.replace(' ', '')

# Build ABFSS URIs to controller lakehouse Files area (point to directory for Spark reads)
source_current_parquet_dir = f"abfss://{container}@{abfss_account}/{controller_lakehouse_name}.Lakehouse/Files/{dataset_name}current"
source_updates_parquet = f"abfss://{container}@{abfss_account}/{controller_lakehouse_name}.Lakehouse/Files/{dataset_name}updates/updates.parquet"

# Targets (fallback to same names as ingest_data)
target_lakehouse = params.get('target_lakehouse', 'BenchmarkLakehouse')
target_warehouse = params.get('target_warehouse', 'BenchmarkWarehouse')

# Staging naming for warehouse flows
staging_table = f"{AZURE_SQL_SCHEMA}.dbo.stg_{sanitized_name}"

print(f"Loaded params: test_case={test_case_name} dataset={dataset_name} source={source} format={fmt} strategy={update_strategy}")
print('Source ABFSS paths:', source_current_parquet_dir, source_updates_parquet)

# Expose as globals for later cells
globals().update({
    'test_case_name': test_case_name,
    'dataset_name': dataset_name,
    'source': source,
    'fmt': fmt,
    'update_strategy': update_strategy,
    'AZURE_SQL_SERVER': AZURE_SQL_SERVER,
    'AZURE_SQL_DB': AZURE_SQL_DB,
    'AZURE_SQL_SCHEMA': AZURE_SQL_SCHEMA,
    'source_current_parquet_dir': source_current_parquet_dir,
    'source_updates_parquet': source_updates_parquet,
    'target_lakehouse': target_lakehouse,
    'target_warehouse': target_warehouse,
    'target_table': target_table,
    'staging_table': staging_table,
    'notes': notes
})


In [ ]:
# Metrics helper (append a single-row metrics entry to target_lakehouse.metrics)
metrics_schema = StructType([
    StructField("test_case_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("source", StringType(), True),
    StructField("format", StringType(), True),
    StructField("rows", IntegerType(), True),
    StructField("update_strategy", StringType(), True),
    StructField("action", StringType(), True),
    StructField("ingest_time_s", FloatType(), True),
    StructField("spinup_time_s", FloatType(), True),
    StructField("query_type", StringType(), True),
    StructField("query_time_s", FloatType(), True),
    StructField("notes", StringType(), True)
])
def append_metrics(action, rows, ingest_time_s, spinup_time_s=0.0, query_type='N/A', query_time_s=float('nan'), notes_local=None):
    notes_local = notes_local if notes_local is not None else notes
    row = [(test_case_name, datetime.now(), source, fmt.upper(), int(rows), update_strategy.title(), action, float(ingest_time_s), float(spinup_time_s), query_type, float(query_time_s), notes_local)]
    spark.createDataFrame(row, schema=metrics_schema).write.mode('append').saveAsTable(f"{target_lakehouse}.metrics")
    print('Metrics appended for action=', action)
print('Metrics helper ready')


In [ ]:
# Helper cell: canonicalize ts_* columns to legacy Spark TimestampType
from pyspark.sql.functions import col

def canonicalize_timestamps(df, prefix="ts_"):
    """
    Cast any columns starting with prefix to legacy Spark TimestampType ('timestamp').
    Returns the DataFrame (unchanged if no ts_* columns present).
    """
    ts_cols = [c for c in df.columns if c.startswith(prefix)]
    if not ts_cols:
        return df
    print("Canonicalizing timestamp columns to legacy Spark TimestampType:", ts_cols)
    for c in ts_cols:
        df = df.withColumn(c, col(c).cast("timestamp"))
    return df

print('canonicalize_timestamps helper ready')


In [ ]:
# SQL helpers: token + connection reused from ingest_data patterns (mssparkutils is available as per note)
from notebookutils import mssparkutils
SQL_COPT_SS_ACCESS_TOKEN = 1256
import struct
import pyodbc

def _token_struct():
    t = mssparkutils.credentials.getToken('https://database.windows.net/')
    exptoken = b''.join(bytes([c]) + b'\x00' for c in t.encode('utf-8'))
    return struct.pack('=i', len(exptoken)) + exptoken

def _pyodbc_conn_with_retry(server=None, database=None, timeout=120, retries=2, backoff=2):
    server = server or AZURE_SQL_SERVER
    database = database or AZURE_SQL_DB
    if not server or not database:
        raise RuntimeError('AZURE_SQL_SERVER and AZURE_SQL_DB must be set (or passed in)')
    if not server.lower().endswith('.database.windows.net'):
        server = server.rstrip('.') + '.database.windows.net'
    conn_str = (
        'Driver={ODBC Driver 18 for SQL Server};'
        f'Server=tcp:{server},1433;'
        f'Database={database};'
        'Encrypt=yes;TrustServerCertificate=no;'
    )
    last_exc = None
    for attempt in range(1, retries + 1):
        try:
            return pyodbc.connect(conn_str, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: _token_struct()}, timeout=timeout)
        except Exception as e:
            last_exc = e
            if attempt < retries:
                time.sleep(backoff * attempt)
            else:
                raise
    raise last_exc

print('_pyodbc_conn_with_retry helper ready')

# Open a connection/cursor if SQL source or warehouse destination is used (happy-path reuse)
conn = None
cur = None
if source == 'sql' or fmt == 'warehouse':
    assert AZURE_SQL_SERVER and AZURE_SQL_DB, 'AZURE_SQL_SERVER and AZURE_SQL_DB required for SQL source or warehouse destination'
    conn = _pyodbc_conn_with_retry(server=AZURE_SQL_SERVER, database=AZURE_SQL_DB)
    cur = conn.cursor()
    print('Opened SQL connection')


## Full Refresh — parquet source
Read the canonical current parquet snapshot from the controller lakehouse Files/<dataset>current directory and overwrite the destination (Delta or Warehouse).

In [ ]:
if update_strategy not in ('full refresh', 'full_refresh', 'fullrefresh') or source != 'lakehouse':
    print('Skipping parquet Full Refresh cell (strategy != full refresh or source != lakehouse)')
else:
    assert source == 'lakehouse', 'Parquet Full Refresh requires source=lakehouse'
    print(f"Starting Full Refresh (parquet -> {fmt}) for test_case: {test_case_name}")
    t0 = time.time()
    # Read from controller lakehouse Files/<dataset>current directory (Spark reads dir of part files)
    df_src = spark.read.parquet(str(source_current_parquet_dir))
    # canonicalize timestamp columns to legacy Spark TimestampType to avoid NTZ issues
    df_src = canonicalize_timestamps(df_src)
    src_count = df_src.count()
    print('Read source_current rows=', src_count)

    if fmt == 'delta':
        table_full = f"{target_lakehouse}.{target_table}"
        print('Writing Delta table ->', table_full)
        df_src.write.mode('overwrite').saveAsTable(table_full)
    elif fmt == 'warehouse':
        table_full = f"{target_warehouse}.dbo.{target_table}"
        print('Writing Warehouse table ->', table_full)
        # ensure nullable update_type if present
        df_write = make_columns_nullable(df_src, columns=[c for c in df_src.columns if c == 'update_type'])
        df_write.write.mode('overwrite').synapsesql(table_full)
    else:
        raise SystemExit(f'Unsupported destination format: {fmt}')

    t1 = time.time()
    ingest_time = t1 - t0
    append_metrics('full_refresh_write', rows=src_count, ingest_time_s=ingest_time, notes_local='parquet source full refresh')
    print(f'Full Refresh complete: rows_written={src_count} ingest_time_s={ingest_time:.2f}')


## Full Refresh — SQL source
Read the canonical current table from Azure SQL and overwrite the destination. (No staging for Delta per rules; for Warehouse we write via synapsesql.)

In [ ]:
if update_strategy not in ('full refresh', 'full_refresh', 'fullrefresh') or source != 'sql':
    print('Skipping SQL Full Refresh cell (strategy != full refresh or source != sql)')
else:
    assert source == 'sql', 'SQL Full Refresh requires source=sql'
    print(f"Starting Full Refresh (sql -> {fmt}) for test_case: {test_case_name}")
    t0 = time.time()
    # Read current table from SQL into pandas then spark (happy-path)
    table_name_sql = f"{AZURE_SQL_SCHEMA}.current_{dataset_name}"
    print(f'Reading SQL table: {table_name_sql} from {AZURE_SQL_SERVER}/{AZURE_SQL_DB}')
    pdf = pd.read_sql(f'SELECT * FROM {table_name_sql}', conn)
    df_src = spark.createDataFrame(pdf)
    # canonicalize timestamp columns to legacy Spark TimestampType to avoid NTZ issues
    df_src = canonicalize_timestamps(df_src)
    src_count = df_src.count()
    print('Read source_current rows (from SQL)=', src_count)

    if fmt == 'delta':
        table_full = f"{target_lakehouse}.{target_table}"
        print('Writing Delta table ->', table_full)
        df_src.write.mode('overwrite').saveAsTable(table_full)
    elif fmt == 'warehouse':
        table_full = f"{target_warehouse}.dbo.{target_table}"
        print('Writing Warehouse table ->', table_full)
        df_write = make_columns_nullable(df_src, columns=[c for c in df_src.columns if c == 'update_type'])
        df_write.write.mode('overwrite').synapsesql(table_full)
    else:
        raise SystemExit(f'Unsupported destination format: {fmt}')

    t1 = time.time()
    ingest_time = t1 - t0
    append_metrics('full_refresh_write_sql', rows=src_count, ingest_time_s=ingest_time, notes_local='sql source full refresh')
    print(f'Full Refresh (SQL source) complete: rows_written={src_count} ingest_time_s={ingest_time:.2f}')


## Full Compare — parquet source
Compare source_current parquet to destination snapshot and append classified event rows to the destination event-log.

In [ ]:
if update_strategy not in ('full compare', 'full_compare', 'fullcompare') or source != 'lakehouse':
    print('Skipping parquet Full Compare cell (strategy != full compare or source != lakehouse)')
else:
    assert source == 'lakehouse', 'Parquet Full Compare requires source=lakehouse'
    print(f"Starting Full Compare (parquet -> {fmt}) for test_case: {test_case_name}")
    t0 = time.time()
    df_src = spark.read.parquet(str(source_current_parquet_dir))
    # canonicalize timestamp columns to legacy Spark TimestampType to avoid NTZ issues
    df_src = canonicalize_timestamps(df_src)
    print('Source rows=', df_src.count())

    # Read destination snapshot depending on destination format
    if fmt == 'delta':
        df_dest = spark.table(f"{target_lakehouse}.{target_table}")
    elif fmt == 'warehouse':
        # read via JDBC into spark (happy-path use pandas fallback for parity with other notebooks)
        table_sql = f"{target_warehouse}.dbo.{target_table}"
        pdf_dest = pd.read_sql(f'SELECT * FROM {table_sql}', conn)
        df_dest = spark.createDataFrame(pdf_dest)
        df_dest = canonicalize_timestamps(df_dest)
    else:
        raise SystemExit(f'Unsupported destination format: {fmt}')
    print('Destination rows=', df_dest.count())

    # Classify changes: inserts, updates, deletes, anomalies
    # Inserts: in source but not in dest
    inserts = df_src.join(df_dest.select('id'), on='id', how='left_anti').withColumn('update_type', lit('insert'))
    # Deletes: in dest but not in source -> append dest row but mark delete
    deletes = df_dest.join(df_src.select('id'), on='id', how='left_anti').withColumn('update_type', lit('delete'))
    # Potential updates: in both
    both = df_src.alias('s').join(df_dest.alias('d'), on='id', how='inner')
    updates = both.filter(col('s.ts_1') > col('d.ts_1'))
    updates = updates.select('s.*').withColumn('update_type', lit('update'))
    anomalies = both.filter(col('d.ts_1') > col('s.ts_1'))
    anomaly_count = anomalies.count()
    if anomaly_count > 0:
        print(f'WARNING: anomalies detected where destination.ts_1 > source.ts_1: count={anomaly_count}')

    # Build events DF: append inserts + updates + deletes (make sure schemas align)
    # Ensure delete rows have same columns as source: use dest row (we set update_type above)
    events = inserts.unionByName(updates, allowMissingColumns=True).unionByName(deletes, allowMissingColumns=True)
    events_count = events.count()
    print('Total events to append=', events_count, '| inserts=', inserts.count(), 'updates=', updates.count(), 'deletes=', deletes.count())

    # Write events to destination event-log
    if fmt == 'delta':
        events.write.mode('append').saveAsTable(f"{target_lakehouse}.{target_table}")
    else:
        events_write = make_columns_nullable(events, columns=[c for c in events.columns if c == 'update_type'])
        events_write.write.mode('append').synapsesql(f"{target_warehouse}.dbo.{target_table}")

    t1 = time.time()
    ingest_time = t1 - t0
    append_metrics('full_compare_append', rows=events_count, ingest_time_s=ingest_time, notes_local='parquet source full compare')
    print('Full Compare parquet complete: events_appended=', events_count, 'ingest_time_s=', ingest_time)


## Full Compare — SQL source
When destination is Delta: do Spark compare (read via pandas->spark). When destination is Warehouse: stage source to staging table and run server-side classification (single server-side T-SQL set of INSERTs).

In [ ]:
if update_strategy not in ('full compare', 'full_compare', 'fullcompare') or source != 'sql':
    print('Skipping SQL Full Compare cell (strategy != full compare or source != sql)')
else:
    assert source == 'sql', 'SQL Full Compare requires source=sql'
    print(f"Starting Full Compare (sql -> {fmt}) for test_case: {test_case_name}")
    # Read source_current from SQL into pandas then spark
    t0 = time.time()
    table_name_sql = f"{AZURE_SQL_SCHEMA}.current_{dataset_name}"
    print(f'Reading SQL table: {table_name_sql}')
    pdf_src = pd.read_sql(f'SELECT * FROM {table_name_sql}', conn)
    df_src = spark.createDataFrame(pdf_src)
    # canonicalize timestamp columns to legacy Spark TimestampType to avoid NTZ issues
    df_src = canonicalize_timestamps(df_src)
    print('Source rows=', df_src.count())

    if fmt == 'delta':
        # read destination snapshot
        df_dest = spark.table(f"{target_lakehouse}.{target_table}")
        print('Destination rows=', df_dest.count())
        both = df_src.alias('s').join(df_dest.alias('d'), on='id', how='inner')
        inserts = df_src.join(df_dest.select('id'), on='id', how='left_anti').withColumn('update_type', lit('insert'))
        deletes = df_dest.join(df_src.select('id'), on='id', how='left_anti').withColumn('update_type', lit('delete'))
        updates = both.filter(col('s.ts_1') > col('d.ts_1')).select('s.*').withColumn('update_type', lit('update'))
        anomalies = both.filter(col('d.ts_1') > col('s.ts_1'))
        anomaly_count = anomalies.count()
        if anomaly_count > 0:
            print(f'WARNING: anomalies detected where dest.ts_1 > src.ts_1: count={anomaly_count}')
        events = inserts.unionByName(updates, allowMissingColumns=True).unionByName(deletes, allowMissingColumns=True)
        events_count = events.count()
        events.write.mode('append').saveAsTable(f"{target_lakehouse}.{target_table}")
        t1 = time.time()
        append_metrics('full_compare_sql_to_delta', rows=events_count, ingest_time_s=(t1-t0), notes_local='sql source -> delta full compare')
        print('Full Compare SQL->Delta complete: events_appended=', events_count)

    elif fmt == 'warehouse':
        # Stage source into warehouse staging table then run server-side T-SQL to classify and append events
        print('Staging source to warehouse staging table ->', staging_table)
        # overwrite staging with df_src
        df_src_write = make_columns_nullable(df_src, columns=[c for c in df_src.columns])
        df_src_write.write.mode('overwrite').synapsesql(staging_table)
        staged_count = df_src.count()
        print('Staged rows=', staged_count)

        # Build column lists for dynamic T-SQL (happy-path: rely on df_src.columns)
        cols = df_src.columns
        col_list = ', '.join(f'[{c}]' for c in cols)
        select_s_cols = ', '.join(f's.[{c}]' for c in cols)
        select_t_cols = ', '.join(f't.[{c}]' for c in cols)

        # Compose T-SQL statements to INSERT classified rows into target event-log table
        target_full = f"{target_warehouse}.dbo.{target_table}"
        sql_statements = []
        # Inserts: in staging not in target
        sql_statements.append(f"INSERT INTO {target_full} ({col_list}, [update_type]) SELECT {select_s_cols}, 'insert' FROM {staging_table} s LEFT JOIN {target_full} t ON s.id = t.id WHERE t.id IS NULL;")
        # Updates: in both and staging.ts_1 > target.ts_1
        sql_statements.append(f"INSERT INTO {target_full} ({col_list}, [update_type]) SELECT {select_s_cols}, 'update' FROM {staging_table} s JOIN {target_full} t ON s.id = t.id WHERE s.ts_1 > t.ts_1;")
        # Deletes: in target not in staging -> append target row with update_type='delete'
        sql_statements.append(f"INSERT INTO {target_full} ({col_list}, [update_type]) SELECT {select_t_cols}, 'delete' FROM {target_full} t LEFT JOIN {staging_table} s ON t.id = s.id WHERE s.id IS NULL;")
        # Anomaly count (dest newer than source)
        sql_statements.append(f"SELECT COUNT(1) AS anomaly_count FROM {target_full} t JOIN {staging_table} s ON t.id = s.id WHERE t.ts_1 > s.ts_1;")

        # Execute the statements sequentially and capture anomaly count from the final SELECT
        print('Executing server-side classification SQL...')
        for i, stmt in enumerate(sql_statements):
            cur.execute(stmt)
            # If this was the final SELECT, fetch anomaly_count
            if stmt.strip().upper().startswith('SELECT'):
                row = cur.fetchone()
                anomaly_count = row[0] if row else 0
                if anomaly_count > 0:
                    print(f'WARNING: anomalies detected (dest.ts_1 > src.ts_1): {anomaly_count}')
        conn.commit()

        # Cleanup staging
        print('Dropping staging table:', staging_table)
        cur.execute(f'DROP TABLE IF EXISTS {staging_table};')
        conn.commit()

        t1 = time.time()
        ingest_time = t1 - t0
        # For metrics we approximate rows appended as staged_count (server statements appended rows internally)
        append_metrics('full_compare_sql_to_warehouse', rows=staged_count, ingest_time_s=ingest_time, notes_local='sql source -> warehouse full compare (server-side)')
        print('Full Compare SQL->Warehouse complete: staged_rows=', staged_count)

    else:
        raise SystemExit(f'Unsupported destination format: {fmt}')


## Incremental — append updates
Append the updates snapshot from source (parquet or SQL updates table) to the destination event-log table. No compare, simple append.

In [ ]:
if update_strategy not in ('incremental', 'increment', 'incremental_update') or source not in ('lakehouse','sql'):
    print('Skipping Incremental cell (strategy != incremental or unsupported source)')
else:
    print(f"Starting Incremental append ({source} -> {fmt}) for test_case: {test_case_name}")
    t0 = time.time()
    if source == 'lakehouse':
        # read updates parquet from controller lakehouse Files/<dataset>updates/updates.parquet
        df_updates = spark.read.parquet(str(source_updates_parquet))
        # canonicalize timestamp columns to legacy Spark TimestampType to avoid NTZ issues
        df_updates = canonicalize_timestamps(df_updates)
    elif source == 'sql':
        table_updates = f"{AZURE_SQL_SCHEMA}.updates_{dataset_name}"
        pdf_upd = pd.read_sql(f'SELECT * FROM {table_updates}', conn)
        df_updates = spark.createDataFrame(pdf_upd)
        df_updates = canonicalize_timestamps(df_updates)
    else:
        raise SystemExit(f'Unsupported source for incremental: {source}')

    upd_count = df_updates.count()
    print('Updates rows to append=', upd_count)

    if fmt == 'delta':
        df_updates.write.mode('append').saveAsTable(f"{target_lakehouse}.{target_table}")
    elif fmt == 'warehouse':
        df_updates_write = make_columns_nullable(df_updates, columns=[c for c in df_updates.columns if c == 'update_type'])
        df_updates_write.write.mode('append').synapsesql(f"{target_warehouse}.dbo.{target_table}")
    else:
        raise SystemExit(f'Unsupported destination format: {fmt}')

    t1 = time.time()
    ingest_time = t1 - t0
    append_metrics('incremental_append', rows=upd_count, ingest_time_s=ingest_time, notes_local='incremental append')
    print(f'Incremental append complete: rows_appended={upd_count} ingest_time_s={ingest_time:.2f}')


## Cleanup & summary
Drop staging if left behind and print the run summary.

In [ ]:
print('\nRun summary:')
print(f' test_case: {test_case_name} | dataset: {dataset_name} | source: {source} | format: {fmt} | strategy: {update_strategy}')

# Ensure staging dropped if present (happy-path, run only if staging exists)
if conn is not None and cur is not None:
    try:
        cur.execute(f"DROP TABLE IF EXISTS {staging_table};")
        conn.commit()
        print('Ensured staging table dropped:', staging_table)
    except Exception:
        print('Staging drop may have failed or staging not used (continuing)')
    finally:
        cur.close()
        conn.close()
        print('Closed SQL connection')

print('Apply updates run complete.')
